## Вариант 5

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import uniform
import matplotlib.pyplot as plt
import tqdm.notebook as tqdm
from scipy.stats import norm

Будем рассматривать нормальное распределение с параметрами $(mean=0, std=2)$

### a) Исследование асимптотической несмещенности $\hat{\sigma^2}$

Для вычисления $EX_{(i)}$ генерируется $k$ выборок, каждая из которых сортируется, а затем берется среднее $i$-ых элементов.

И аналогично вычисляются $EX_{(i)}^2$ 

In [22]:
def get_mean_of_std_star(loc, scale, n, k):
    """
    Function calculate E[std^2] by formula below
    E[std^2] = E[(1/(2n)) * sum(X_{2i} - X_{2i-1})^2] = 
    = (1/(2n)) * sum(E[X_{2i}]^2 - EX_{2i}EX_{2i-1} + E[X_{2i-1}]^2)
    """
    # generate random points
    data = norm.rvs(loc=loc, scale=scale, size = (k, n))
    data_sorted = np.sort(data, axis=1)
    data_sorted_squered = data_sorted ** 2
    mean_ordered_stat = data_sorted.mean(axis=0)
    mean_ordered_squered_stat = data_sorted_squered.mean(axis=0)
    stat = 0
    half_n = int(n / 2)
    for i in range(0, half_n):
        stat += mean_ordered_squered_stat[2 * i + 1] -\
            mean_ordered_stat[2 * i + 1] * mean_ordered_stat[2 * i] +\
            mean_ordered_squered_stat[2 * i]
    return stat / n

In [21]:
true_mean = 0
true_std = 2
step = 2
n = 100
k = 1000
for i in range(12):
    mean_of_std_star = get_mean_of_std_star(loc=true_mean, scale=true_std, n=n, k=k)
    print(f'|std - std_star| = {abs(true_std - mean_of_std_star)}, n = {n}')
    n *= step

|std - std_star| = 0.061092288612941026, n = 100
|std - std_star| = 0.04467408888647917, n = 200
|std - std_star| = 0.009016597088159717, n = 400
|std - std_star| = 0.008575512915684147, n = 800
|std - std_star| = 0.0024399179975964636, n = 1600
|std - std_star| = 0.0019819170500796623, n = 3200
|std - std_star| = 0.0013185067836727349, n = 6400
|std - std_star| = 0.001021458592344926, n = 12800
|std - std_star| = 0.0006541098218817432, n = 25600
|std - std_star| = 0.0001016129183319947, n = 51200
|std - std_star| = 0.00044337008570405345, n = 102400
|std - std_star| = 5.766209399560296e-05, n = 204800


Видно, что с ростом $n$ модуль разницы между дисперсией и мат.ожиданием оценкой дисперсии стремится к нулю, что указывает асимптотическую несмещенность

### б) Исследование состоятельности $\hat{\sigma^2}$

$
E\hat{\sigma^2} \rightarrow^P \sigma^2 ~\Leftrightarrow~ P(|\hat{\sigma^2} - \sigma^2| \ge \epsilon) \le \frac{E|\hat{\sigma^2} - \sigma^2|}{\epsilon} \approx \frac{\frac{1}{m}\sum_{1 \le i \le m }{|\hat{\sigma_i^2} - \sigma^2|}}{\epsilon}
$

In [24]:
M = 10
true_mean = 0
true_std = 2
step = 2
eps = 0.1
n = 100
k = 1000
for i in range(12):
    stats = np.zeros(M)
    for i in range(M):
        mean_of_std_star = get_mean_of_std_star(loc=true_mean, scale=true_std, n=n, k=k)
        stats[i] = abs(true_std - mean_of_std_star)
    mean = stats.mean()
    print(f't = {mean / eps}, n = {n}')
    n *= step

t = 0.5919577371086739, n = 100
t = 0.36482346959235284, n = 200
t = 0.1572412229935711, n = 400
t = 0.0835466609760207, n = 800
t = 0.033638009342958235, n = 1600
t = 0.01160287126621373, n = 3200
t = 0.016402170891721024, n = 6400
t = 0.0075396653795090085, n = 12800
t = 0.004863594187515696, n = 25600
t = 0.0030088219940660466, n = 51200
t = 0.0019096449748077227, n = 102400
t = 0.0013857358549864693, n = 204800


Как видно с ростом $n$ величина $t$ стремится к нулю, что указывает на состоятельность оценки.